# Prepare environment

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import data

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

baskets = pd.read_csv('/kaggle/input/groceries/groceries - groceries.csv')
baskets.head()

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Prepare data

In [52]:
# One hot encoding item
basket_sets = pd.get_dummies(baskets.drop(columns=['Item(s)'], axis=1), prefix="item")
basket_sets = basket_sets.groupby(level=0, axis=1).sum()
basket_sets

,item_Instant food products,item_UHT-milk,item_abrasive cleaner,item_artif. sweetener,item_baby cosmetics,item_baby food,item_bags,item_baking powder,item_bathroom cleaner,item_beef,...,item_turkey,item_vinegar,item_waffles,item_whipped/sour cream,item_whisky,item_white bread,item_white wine,item_whole milk,item_yogurt,item_zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
9831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Apply association rule

In [79]:
# Filter items with support more than 3%
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.033452,(item_UHT-milk)
1,0.052466,(item_beef)
2,0.033249,(item_berries)
3,0.080529,(item_bottled beer)
4,0.110524,(item_bottled water)
...,...,...
58,0.048907,"(item_root vegetables, item_whole milk)"
59,0.040061,"(item_whole milk, item_soda)"
60,0.042298,"(item_whole milk, item_tropical fruit)"
61,0.032232,"(item_whole milk, item_whipped/sour cream)"


In [80]:
# List the top100 of itemsets with strong association
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(item_whole milk),(item_bottled water),0.255516,0.110524,0.034367,0.134501,1.216940,0.006126,1.027703
1,(item_bottled water),(item_whole milk),0.110524,0.255516,0.034367,0.310948,1.216940,0.006126,1.080446
2,(item_whole milk),(item_citrus fruit),0.255516,0.082766,0.030503,0.119379,1.442377,0.009355,1.041577
3,(item_citrus fruit),(item_whole milk),0.082766,0.255516,0.030503,0.368550,1.442377,0.009355,1.179008
4,(item_rolls/buns),(item_other vegetables),0.183935,0.193493,0.042603,0.231620,1.197047,0.007013,1.049620
5,(item_other vegetables),(item_rolls/buns),0.193493,0.183935,0.042603,0.220179,1.197047,0.007013,1.046477
6,(item_root vegetables),(item_other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
7,(item_other vegetables),(item_root vegetables),0.193493,0.108998,0.047382,0.244877,2.246605,0.026291,1.179941
8,(item_tropical fruit),(item_other vegetables),0.104931,0.193493,0.035892,0.342054,1.767790,0.015589,1.225796
9,(item_other vegetables),(item_tropical fruit),0.193493,0.104931,0.035892,0.185497,1.767790,0.015589,1.098913
